In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import pickle

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
batch_size = 32
block_size = 128
max_iters = 3000
learning_rate = 3e-4
eval_iters = 50
n_embd = 384
n_head = 4
n_layer = 4
dropout = 0.2

cpu


In [2]:
# Open the text file and read its contents
with open('wizard_of_oz.txt','r',encoding='utf-8') as f:
    text = f.read()

# Create a sorted list of unique characters in the text
chars = sorted(list(set(text)))
print(chars)  # Print the list of unique characters
print(len(chars))  # Print the number of unique characters

# Define the vocabulary size as the number of unique characters
vocab_size = len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']
81


In [3]:
# Create dictionaries to map characters to integers and vice versa
string_to_int = {ch: i for i, ch in enumerate(chars)}
int_to_string = {i: ch for i, ch in enumerate(chars)}

# Define encoding and decoding functions using lambda expressions
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

# Encode the string 'hello' to a list of integers
encoded_hello = encode('hello')
# Decode the list of integers back to the string 'hello'
decoded_hello = decode(encoded_hello)

# Print the encoded list of integers
print(encoded_hello)


[61, 58, 65, 65, 68]


In [4]:
print(decoded_hello)

hello


In [5]:
data = torch.tensor(encode(text), dtype=torch.long)

In [6]:
data[:100]

tensor([80,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,
         1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26,
        49,  0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,
         0,  0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1,
        36, 25, 38, 28,  1, 39, 30,  1, 39, 50])

In [7]:
# Split the data into training and validation sets
n = int(0.8 * len(data))
train_data = data[:n]
val_data = data[n:]

# Function to get a batch of data
def get_batch(split):
    # Select the appropriate dataset
    data = train_data if split == 'train' else val_data
    # Randomly select indices for the batch
    ix = torch.randint(len(data) - block_size, (batch_size,))
    # Create input and target tensors
    x = torch.stack([data[i:i + block_size] for i in ix])
    y = torch.stack([data[i + 1:i + block_size + 1] for i in ix])
    # Move tensors to the appropriate device
    x, y = x.to(device), y.to(device)
    return x, y

x,y = get_batch('train')
print('inputs:')
print(x)
print('outputs:')
print(y)

inputs:
tensor([[57, 72,  1,  ...,  9,  3,  1],
        [63, 74, 72,  ..., 65, 65, 11],
        [68, 62, 67,  ..., 73, 61, 58],
        ...,
        [76, 54, 73,  ..., 58,  1, 66],
        [57,  1, 66,  ..., 58, 72,  1],
        [ 1, 32, 74,  ..., 62, 60, 61]])
outputs:
tensor([[72,  1, 73,  ...,  3,  1, 69],
        [74, 72, 73,  ..., 65, 11,  3],
        [62, 67, 73,  ..., 61, 58,  1],
        ...,
        [54, 73, 58,  ...,  1, 66, 54],
        [ 1, 66, 54,  ..., 72,  1, 56],
        [32, 74, 60,  ..., 60, 61, 73]])


In [8]:
@torch.no_grad()  # Disable gradient calculation for this function
def estimate_loss():
    out = {}  # Dictionary to store the average loss for 'train' and 'val' splits
    model.eval()  # Set the model to evaluation mode
    for split in ['train', 'val']:  # Iterate over both training and validation splits
        losses = torch.zeros(eval_iters)  # Tensor to store losses for each iteration
        for k in range(eval_iters):  # Loop over the number of evaluation iterations
            X, Y = get_batch(split)  # Get a batch of data
            logits, loss = model(X, Y)  # Perform a forward pass and compute the loss
            losses[k] = loss.item()  # Store the loss value
        out[split] = losses.mean()  # Compute the average loss for the split
    model.train()  # Set the model back to training mode
    return out  # Return the dictionary containing average losses

In [9]:
class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer("tril", torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)
        q = self.query(x)
        
        wei = q @ k.transpose(-2, -1) * k.shape[-1] ** -0.5
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)

        v = self.value(x)
        out = wei @ v
        return out

class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedForward(nn.Module):
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )
    
    def forward(self, x):
        return self.net(x)
    
class Block(nn.Module):
    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)
    
    def forward(self, x):
        y = self.sa(x)
        x = self.ln1(x + y)
        y = self.ffwd(x)
        x = self.ln2(x + y)
        return x

class GPTLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        # Initialize an embedding layer that maps each token to a vector of size vocab_size
        self.token_embedding = nn.Embedding(vocab_size, n_embd)
        self.position_embedding = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head = n_head) for _ in range(n_layer)])

        self.ln_f = nn.LayerNorm(n_embd)
        self.ln_head = nn.Linear(n_embd, vocab_size)

        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
          
    
    def forward(self, index, targets=None):
        # Get the logits by passing the input indices through the embedding layer
        B, T = index.shape

        tok_emb = self.token_embedding(index)
        pos_emb = self.position_embedding(torch.arange(T, device=device))
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.ln_head(x)

        # If targets are not provided, return the logits and None for loss
        if targets is None:
            return logits, None
        
        # Reshape logits and targets for computing cross-entropy loss
        B, T, C = logits.shape
        logits = logits.view(B * T, C)
        targets = targets.view(B * T)
        loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # Generate new tokens up to max_new_tokens
        for _ in range(max_new_tokens):
            index_cond = index[:, -block_size:]
            # Get the logits for the current index
            logits, loss = self.forward(index_cond)
            # Consider only the logits of the last time step
            logits = logits[:, -1, :]
            # Apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1)
            # Sample the next token from the probability distribution
            index_next = torch.multinomial(probs, num_samples=1)
            # Concatenate the new token to the existing sequence
            index = torch.cat([index, index_next], dim=-1)

        return index

In [10]:
# Initialize the model with the vocabulary size
model = GPTLanguageModel(vocab_size)

# Move the model to the appropriate device (CPU or GPU)
m = model.to(device)



In [11]:
# Initialize the optimizer with model parameters and learning rate
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

# Training loop
for iter in range(max_iters):
    # Evaluate and print loss every eval_iters iterations
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f'Iter {iter}, train loss: {losses["train"]:.3f}, val loss: {losses["val"]:.3f}')

    # Get a batch of training data
    xb, yb = get_batch('train')

    # Perform a forward pass and compute the loss
    logits, loss = model.forward(xb, yb)
    
    # Zero the gradients before backpropagation
    optimizer.zero_grad(set_to_none=True)
    
    # Backpropagate the loss
    loss.backward()
    
    # Update the model parameters
    optimizer.step()

print(loss.item())

Iter 0, train loss: 4.436, val loss: 4.436
Iter 50, train loss: 2.509, val loss: 2.554
Iter 100, train loss: 2.423, val loss: 2.484


KeyboardInterrupt: 

In [ ]:
with open('model.pkl', 'wb') as f:
    pickle.dump(model, f)
print('Model saved to model.pkl')

In [ ]:
prompt = "Dorothy lived in the midst of the great Kansas prairies, with Uncle Henry, who was a farmer, and Aunt Em, who was the farmer's wife."

# Create a context tensor with a single element (0) on the appropriate device
context = torch.tensor(encode(prompt), dtype=torch.long, device=device)

# Generate 500 new tokens using the model and decode them to a string
generated_chars = decode(m.generate(context.unsqueeze(0), max_new_tokens = 100)[0].tolist())

# Print the generated characters
print(generated_chars)